In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from qiskit_metal import draw, Dict,designs
from qiskit_metal.qlibrary.core import BaseQubit
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.draw import LineString
# import qiskit_metal as metal
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.resonator.readoutres_fc import ReadoutResFC
from  qiskit_metal.qlibrary.user_components.my_qcomponent import  New_Transomon_Cross
import  math

In [9]:
# Initialise design
design = designs.DesignPlanar()
# Specify design name
design.metadata['design_name'] = 'FlipChip_Device'
# launch GUI
gui = MetalGUI(design)
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

In [10]:
# delete existing components
design.delete_all_components()

q1_x = '0.0 mm'
q1_y = '0.0 mm'
q1 = New_Transomon_Cross(design, 'Q1', options = Dict(pos_x=q1_x, pos_y=q1_y))



In [11]:
q1.options.gds_cell_name = 'FakeJunction_01'
q1.options

{'pos_x': '0.0 mm',
 'pos_y': '0.0 mm',
 'orientation': '0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {},
 'cross_width': '1340um',
 'cross_height': '80um',
 'cross_inside_width': '20um',
 'pad_width': '40um',
 'pad_height': '40um',
 'pad_distance': '20um',
 'jj_pad_width': '10um',
 'jj_pad_height': '9um',
 'jj_etch_length': '4um',
 'jj_etch_pad1_width': '4um',
 'jj_etch_pad2_width': '7um',
 'round_corner_radius': '10um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'FakeJunction_01'}

In [12]:
from qiskit_metal.qlibrary.user_components.my_qcomponent import MyReadoutRes01

options = Dict(
       readout_radius='50 um',
       readout_cpw_width='5 um',
       readout_cpw_gap='10 um',
       readout_cpw_turnradius='27 um',
       vertical_start_length = '40 um',
       vertical_end_length = '300 um',
       horizontal_start_length01= '400 um',
       horizontal_start_length02 = '400 um',
       horizontal_end_length = '500 um',
       total_length = '4200 um',
       arc_step='1 um',
       meander_round = '5',
       orientation='0',
       layer='2',
       layer_subtract='2',
       inverse = False,
       mirror = False,
       subtract=True,
       )
location_x = q1.parse_options().cross_width/4
# the resonator is set to have its origin at the center of the circular patch.
# So we set the qubit and the resonator to share the same coordinate (q1_x, q1_y)
r1 = MyReadoutRes01(design, 'R1', options = Dict(pos_x = location_x, pos_y = q1_y, **options))
# rebuild in gui
gui.rebuild()
gui.autoscale()

In [13]:
x,y = r1.make_ro()
x, y=-x,-y
x, y

(-0.9657654975296135, -0.795)

In [14]:
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from collections import  OrderedDict
readout_line_ydistance = 0.15
readout_line_xdistance = 0.5
pad_pos_y0 = y-readout_line_ydistance
pad_pos_x0 = x -readout_line_xdistance
pad_pos_y1 = pad_pos_y0
pad_pos_x1 = x+4*readout_line_xdistance
pin_opt = Dict(hfss_wire_bonds=True, pin_inputs=Dict(start_pin=Dict(component='Launch_Readout0',pin='tie'),
                         end_pin=Dict(component='Launch_Readout1',pin='tie'),),
                          trace_width='10um',trace_gap='6um',layer = '2',)
pin_opt.fillet = '30 um'
pin_opt.lead.start_straight = '90 um'
pin_opt.lead.end_straight = '90 um'

jogsS = OrderedDict()
jogsS[0] = ["R", '90um']
jogsE = OrderedDict()
jogsE[0] = ["L", '90um']

launch_readout0 = LaunchpadWirebond(design,'Launch_Readout0',options=Dict(pos_x=pad_pos_x0,pos_y=pad_pos_y0,orientation='90',layer = '2'))
launch_readout1 = LaunchpadWirebond(design,'Launch_Readout1',options=Dict(pos_x=pad_pos_x1,pos_y=pad_pos_y1,orientation='90',layer = '2'))
t1_readout_q_01=RoutePathfinder(design,'T1_Readout_Q_01',options=pin_opt)
gui.rebuild()
gui.autoscale()


In [15]:
a_gds = design.renderers.gds
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [17]:
a_gds.options['short_segments_to_not_fillet'] = 'True'
scale_fillet = 2.0
a_gds.options['check_short_segments_by_scaling_fillet'] = scale_fillet
a_gds.options.negative_mask = {'main': [1,2]}
a_gds.options.cheese.view_in_file =  {'main': {1: False, 2:False},}
a_gds.options.no_cheese.view_in_file =  {'main': {1: False, 2:False}, }
a_gds.export_to_gds('flip_chip_test_planar.gds')

1